# Importing libriries : 

In [58]:
import os
import cv2
import numpy as np
from imutils import paths
from sklearn.preprocessing import label_binarize
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.models import resnet50
from collections import deque

In [59]:
# Set random seed for reproducibility
torch.manual_seed(42)

# Filtring the data : 

In [60]:
Sport_Labels = set(['boxing', 'swimming', 'table_tennis'])
print("Images are being loaded ... ")   
datapath = '../data'
pathToImages = list(paths.list_images(datapath))
data = []
labels = []

for image_path in pathToImages:
    label = image_path.split(os.path.sep)[-2]
    if label not in Sport_Labels:
        continue
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Corrected the function call
    image = cv2.resize(image, (244, 244))
    data.append(image)
    labels.append(label)

Images are being loaded ... 


In [61]:
data = np.array(data)
labels = np.array(labels)

In [62]:
lb = label_binarize(labels, classes=list(Sport_Labels))
labels = torch.tensor(lb, dtype=torch.float32)

In [63]:
X_train, X_test, Y_train, Y_test = train_test_split(data, labels, test_size=0.25, stratify=labels, random_state=42)

In [64]:
class CustomDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

In [65]:
train_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomRotation(30),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[123.68/255.0, 116.779/255.0, 103.939/255.0], std=[1, 1, 1]),
])

In [66]:
test_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[123.68/255.0, 116.779/255.0, 103.939/255.0], std=[1, 1, 1]),
])

In [67]:
train_dataset = CustomDataset(X_train, Y_train, transform=train_transform)
test_dataset = CustomDataset(X_test, Y_test, transform=test_transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [68]:
class ResNetModel(nn.Module):
    def __init__(self, num_classes):
        super(ResNetModel, self).__init__()
        self.base_model = resnet50(pretrained=True)
        in_features = self.base_model.fc.in_features
        self.base_model.fc = nn.Identity()
        self.fc = nn.Sequential(
            nn.Linear(in_features, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        features = self.base_model(x)
        out = self.fc(features)
        return out

In [69]:
model = ResNetModel(num_classes=len(Sport_Labels))
for param in model.base_model.parameters():
    param.requires_grad = False

# Optimizer and Loss Function
optimizer = optim.SGD(model.fc.parameters(), lr=0.001, momentum=0.9, weight_decay=1e-4)
criterion = nn.CrossEntropyLoss()

In [70]:
def train(model, train_loader, criterion, optimizer, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, torch.argmax(labels, dim=1))
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch [{epoch + 1}/{num_epochs}] Loss: {running_loss / len(train_loader)}")

In [71]:
num_epochs = 10
train(model, train_loader, criterion, optimizer, num_epochs=num_epochs)

Epoch [1/10] Loss: 0.9866215717792511
Epoch [2/10] Loss: 0.6456092482805252


KeyboardInterrupt: 